In [240]:
import bitfinex

import datetime
import time
import os

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

----------

Bitfinex gestisce il tempo in relazione al **UTC** (*Universal Time Convention*), il nostro è chiaramente UTC+1.

In [292]:
api_v1 = bitfinex.bitfinex_v1.api_v1()
pairs = np.array(api_v1.symbols())
pairs = pairs[[(pair[-3:]=='usd' and pair[-4]!=':') for pair in pairs]]

No keys, only access to public API functions


In [293]:
pairs

array(['btcusd', 'ltcusd', 'ethusd', 'etcusd', 'rrtusd', 'zecusd',
       'xmrusd', 'dshusd', 'xrpusd', 'iotusd', 'eosusd', 'sanusd',
       'omgusd', 'neousd', 'etpusd', 'qtmusd', 'avtusd', 'edousd',
       'btgusd', 'datusd', 'qshusd', 'yywusd', 'gntusd', 'sntusd',
       'batusd', 'mnausd', 'funusd', 'zrxusd', 'tnbusd', 'trxusd',
       'rcnusd', 'rlcusd', 'sngusd', 'repusd', 'necusd', 'iosusd',
       'aiousd', 'requsd', 'lrcusd', 'waxusd', 'daiusd', 'bftusd',
       'mtnusd', 'odeusd', 'antusd', 'stjusd', 'xlmusd', 'xvgusd',
       'mkrusd', 'kncusd', 'poausd', 'lymusd', 'utkusd', 'veeusd',
       'orsusd', 'aucusd', 'cbtusd', 'zcnusd', 'ncausd', 'cndusd',
       'ctxusd', 'paiusd', 'essusd', 'hotusd', 'dtausd', 'iqxusd',
       'wprusd', 'zilusd', 'bntusd', 'xrausd', 'manusd', 'dgxusd',
       'vetusd', 'tknusd', 'gotusd', 'xtzusd', 'boxusd', 'mgousd',
       'yggusd', 'mlnusd', 'wtcusd', 'omnusd', 'drnusd', 'pnkusd',
       'dgbusd', 'bsvusd', 'enjusd', 'onlusd', 'rbtusd', 'ustu

In [273]:
save_path = './data/'

for pair in pairs:
    pair_data = hist_data(symb=pair)

    ind = [np.ndim(x) != 0 for x in pair_data]
    pair_data = [i for (i, v) in zip(pair_data, ind) if v]

    names = ['unix_stamp', 'open', 'close', 'high', 'low', 'volume']
    df = pd.DataFrame(pair_data, columns=names)
    df.drop_duplicates(inplace=True)
    df['datetime'] = pd.to_datetime(df['unix_timestamp'], unit='ms')
    
    df.set_index('datetime', inplace=True)
    df.sort_index(inplace=True)

    print('Download complete! Saving to CSV format..')
    df.to_csv(save_path+f'{pair.upper()}.csv')
    print('Done! Moving to next pair.')

print('END')

No keys, only access to public API functions
No keys, only access to public API functions


KeyboardInterrupt: 

----

## Methods declaration

In [214]:
def date_2_stamp(date):
    '''
    Convert UTC date format to unix timestamp.
    
    Parameters
    ----------
    datetime : string
        Object indicating desired date to convert. Suggested format (YYYY-MM-DD).
    '''
    
    stamp = pd.to_datetime(pd.to_datetime(date))
    stamp = ((stamp - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s'))*1000
    
    return stamp

In [272]:
def hist_data(symb='btcusd', start=1364767200000, stop='today', interval='1m', tick_limit=1000, step=60000000):
    api_v2 = bitfinex.bitfinex_v2.api_v2()
    
    if stop == 'today':
        today = datetime.datetime.today()
        stop = time.mktime(today.timetuple())*1000
    
    data = []
    
    print(f'Downloading {pair.upper()}..')
    year = datetime.datetime.fromtimestamp(start/1000).year

    while start < stop:          
        end = start + step
        res = api_v2.candles(symbol=symb, interval=interval, limit=tick_limit, start=start, end=end)
        data.extend(res)
        
        new_year = datetime.datetime.fromtimestamp(end/1000).year
        if new_year > year:
            print(f'{year} completed!')
            year = new_year

        #print(f"{pd.to_datetime(start, unit='ms')} - {pd.to_datetime(end, unit='ms')}\n")
        start += step
        time.sleep(1.5)

    return data

---

In [183]:
t_start = datetime.datetime(2021, 1, 2, 0, 0)
t_start = time.mktime(t_start.timetuple()) * 1000

t_stop = datetime.datetime(2021, 1, 3, 0, 0)
t_stop = time.mktime(t_stop.timetuple()) * 1000

In [180]:
import bitfinex
import pandas as pd
import numpy as np
import datetime
import time
import os


# Create a function to fetch the data
def fetch_data(start=1364767200000, stop=1545346740000, symbol='btcusd', interval='1m', tick_limit=1000, step=60000000):
    # Create api instance
    api_v2 = bitfinex.bitfinex_v2.api_v2()

    data = []
    start = start - step
    while start < stop:

        start = start + step
        end = start + step
        res = api_v2.candles(symbol=symbol, interval=interval, limit=tick_limit, start=start, end=end)
        data.extend(res)
        print('Retrieving data from {} to {} for {}'.format(pd.to_datetime(start, unit='ms'),
                                                            pd.to_datetime(end, unit='ms'), symbol))
        time.sleep(1.5)
    return data


# Define query parameters
bin_size = '1m'
limit = 1000
time_step = 1000 * 60 * limit

t_start = datetime.datetime(2021, 1, 2, 0, 0, tzinfo=datetime.timezone.utc)
t_start = time.mktime(t_start.timetuple()) * 1000

t_stop = datetime.datetime(2021, 1, 3, 0, 0, tzinfo=datetime.timezone.utc)
t_stop = time.mktime(t_stop.timetuple()) * 1000

api_v1 = bitfinex.bitfinex_v1.api_v1()
pair = api_v1.symbols()[0]

save_path = './data'

if os.path.exists(save_path) is False:
    os.mkdir(save_path)


data_list = fetch_data(start=t_start, stop=t_stop, symbol=pair, interval=bin_size, tick_limit=limit, step=time_step)

not_emp = [np.ndim(x) != 0 for x in data_list] 
data_list = [i for (i, v) in zip(data_list, not_emp) if v]

# Create pandas data frame and clean data
names = ['time', 'open', 'close', 'high', 'low', 'volume']
df = pd.DataFrame(pair_data, columns=names)
df.drop_duplicates(inplace=True)
# df['time'] = pd.to_datetime(df['time'], unit='ms')
df.set_index('time', inplace=True)
df.sort_index(inplace=True)

print('Done downloading data. Saving to .csv.')
df.to_csv('{}/bitfinex_{}.csv'.format(save_path, pair))
print('Done saving data. Moving to next pair.')

print('Done retrieving data')

No keys, only access to public API functions
No keys, only access to public API functions
Retrieving data from 2021-01-01 23:00:00 to 2021-01-02 15:40:00 for btcusd
Retrieving data from 2021-01-02 15:40:00 to 2021-01-03 08:20:00 for btcusd
Retrieving data from 2021-01-03 08:20:00 to 2021-01-04 01:00:00 for btcusd


KeyboardInterrupt: 